In [2]:
# Importing the libraries 
import pandas as pd
import re
import os
import zipfile
from statsmodels.stats.proportion import proportion_confint

In [3]:
rootpath = "/hdd/work/d4ad_standardization/"
filepath = "./D4AD_Standardization/data/raw/etpl_all_programsJune3.xls"

columns = [
    "NAME",
    "NAME_1",
    "DESCRIPTION",
    "PREREQUISITES",
    "FEATURESDESCRIPTION",
    "STREET1",
    "CITY",
    "STATE",
    "ZIP",
    "WEBSITE",
    "COUNTY",
    "NONGOVAPPROVAL",
    "STATECOMMENTS",
    "CIPCODE",
    "PROVIDERID",
    "APPROVINGAGENCYID"
]

df = pd.read_excel(rootpath + filepath, usecols=columns)
print('done')

done


In [4]:
# Set up columns to keep, fields, locations for writing
processedpath = "D4AD_Standardization/data/processed/"
externalpath = "D4AD_Standardization/data/external/"

content_is = "NAME_1_lookup.csv"

the_df = df.sample(n=100, random_state=42)

columns_to_save = ['STANDARDIZEDNAME1', 'NAME_1', 'PROVIDERID',
                    'APPROVINGAGENCYID', 'CIPCODE']
print('done')

done


In [11]:
# This should probably be put in a seperate file but the Tools and Technology of
# ONET should basically cover every possible tool/tech and 

# Yeah, and then Career One Stop has a giant list of certs, occupational licenses
# see: https://www.careeronestop.org/Developers/Data/data-downloads.aspx
# need to stand up or use a db connector to import it, though
ONET_TOOLS_TECH_URL_NAME = ("https://www.onetcenter.org/dl_files/database/db_20_1_text/Tools%20and%20Technology.txt", "onet_tools_tech.csv")
CAREERONESTOP_CERTIFICATIONS_URL_NAME = ("https://www.careeronestop.org/TridionMultimedia/tcm24-48614_CareerOnestop_Certifications_07072020.zip", "career_one_stop.zip")

filepath = rootpath + externalpath

for dataset in (ONET_TOOLS_TECH_URL_NAME, CAREERONESTOP_CERTIFICATIONS_URL_NAME):
    url, filename = dataset
    print("running ...", f'\nwget -O {filepath+filename} {url}')
    os.system(f'wget -O {filepath+filename} {url}')
    print("filetype is",  filename[-3:])

    if filename[-3:] == 'zip':
        with zipfile.ZipFile(filepath+filename,"r") as zip_ref:
            zipdir = filepath+filename[:-4]
            print("unzipping {} to ...".format(filename), zipdir)
            os.mkdir(zipdir)
            zip_ref.extractall(zipdir)

# todo: d/l giant list of acronyms off of wikipedia
# here: https://en.wikipedia.org/wiki/Lists_of_abbreviations
# todo: figure out a standardization/needs-standardization logic
#       the needs logic should basically look up into abbreviations or the certification
#           and replace, remove or annotate (like see description, place a star)

running ... 
wget -O /hdd/work/d4ad_standardization/D4AD_Standardization/data/external/onet_tools_tech.csv https://www.onetcenter.org/dl_files/database/db_20_1_text/Tools%20and%20Technology.txt
filetype is csv
running ... 
wget -O /hdd/work/d4ad_standardization/D4AD_Standardization/data/external/career_one_stop.zip https://www.careeronestop.org/TridionMultimedia/tcm24-48614_CareerOnestop_Certifications_07072020.zip
filetype is zip
unzipping career_one_stop.zip to ... /hdd/work/d4ad_standardization/D4AD_Standardization/data/external/career_one_stop


In [4]:
pd.set_option('display.max_rows', None)

# anomalous cases to test against
"""
CXA-300-1I Advanced Administration for Citrix ...	
LEED AP Exam
Accounting Applications I/
CompTIA i-Net+ Certification
(EHR125D) Electronic Health Records Specialist
Patient Care Technician (PCT)
CELW-109-01

A.A.Degree: Liberal Arts/Ele-Sec Educ
(10 months) Some Course
Windows2000

Electrical/Electronic Control System & Transpo...
PowerPoint Level 1
"""
# Can throw into https://regex101.com/ and cook up a regex that gets them all


# quick notes:
# * so everything in parathensis can be removed and, actually, is valuable abbreviation data
# * the description is interesting, it often has some location info that isn't needed. THis might benefit from a NER type approach where we label tokens that should be removed. But that's not here nor there.
# * trailing non-alphanumerics should be removed (like "...tions I/")
# * Somethings, "CEBC-211-01" are too abbreviated (e.g. all caps, )
# * I could run the abbreviation extractor on this but I'm kinda hestitant to mess 
# with another vauge program, however, it might really simplify some of this
# * There are degree, time realated things that don't belong, "A.A.Degree: Liberal Arts/Ele-Sec Educ" or "10-MOS" (10 months), or Paralegal certificate
# * Saw one mashed together thing, Windows2000

# * loaded a giant list of certifications to ./data/external/career_one_stop, it includes canonical names, some abbreviations (and we can generate abbrevations from the canonical name)
# * yeah that list of cert names will be useful, so it looks like typically the course name describes a single thing, we don't list multiple certifications, programs, etc. This enables me to chunk up the ... thign is there aresome liek CEWL that don't show up at all

'\nCXA-300-1I Advanced Administration for Citrix ...\t\nLEED AP Exam\nAccounting Applications I/\nCompTIA i-Net+ Certification\n(EHR125D) Electronic Health Records Specialist\nPatient Care Technician (PCT)\nCELW-109-01\n\nA.A.Degree: Liberal Arts/Ele-Sec Educ\n(10 months) Some Course\nWindows2000\n\nElectrical/Electronic Control System & Transpo...\nPowerPoint Level 1\n'

In [5]:
# A) 
# The program or course name can start or end with a matching parenthesis. In these cases
# we assume that no other matching parenthesis are present and apply an appropriate
# regex for that...

# First, set up standardized column with default values
the_df["STANDARDIZEDNAME_1"] = ""

# ... then extract names for those with opening parens
open_parenthesis_index = the_df.NAME_1.str[0] == '('
open_parenthesis_regex = '''
                (?P<paren>\(.*\)) # get the first parathesis
                (?P<the_name>.*)  # then get the actual name
                '''

the_df.loc[open_parenthesis_index, "STANDARDIZEDNAME_1"] =\
    the_df.loc[open_parenthesis_index, 'NAME_1']\
          .str\
          .extract(open_parenthesis_regex, flags=re.VERBOSE).the_name

# ... then extract names for those with closing parens
close_parenthesis_index = the_df.NAME_1.str[-1] == ')'
closing_parenthesis_regex = '''
                (?P<the_name>.*)  # get the actual name
                (?P<paren>\(.*\)) # get the last parathensis                
                '''
the_df.loc[close_parenthesis_index, "STANDARDIZEDNAME_1"] =\
    the_df.loc[close_parenthesis_index, 'NAME_1']\
          .str\
          .extract(closing_parenthesis_regex, flags=re.VERBOSE).the_name

# ... then we copy over content that has a internal parenthesis with those
# parenthesis removed and ignore everything after, e.g. "ABC (123) DEF" --> "ABC"
internal_parenthesis_index =\
    the_df['NAME_1'].str.contains('\(|\)', regex=True) &\
        ~(close_parenthesis_index|open_parenthesis_index)

the_df.loc[internal_parenthesis_index, "STANDARDIZEDNAME_1"] =\
    the_df.loc[internal_parenthesis_index, 'NAME_1']\
          .str\
          .extract(closing_parenthesis_regex, flags=re.VERBOSE).the_name

# ... finally, just copy over everything else
no_parenthesis_index = ~(close_parenthesis_index |\
                         open_parenthesis_index  |\
                         internal_parenthesis_index)
the_df.loc[no_parenthesis_index, "STANDARDIZEDNAME_1"] =\
    the_df.loc[no_parenthesis_index, 'NAME_1']

the_df[['NAME_1', 'STANDARDIZEDNAME_1']]

# 2)
# So now we have silver version data that has a lot of abbreviations, acroynmns
# that need to be spelled out. We do this with bulk match and replace.

# acroynmns can be done with the career one stop credentials list
# abbreviatins are harder. Let's start on acroynmns first.

# 3) 
# Then go after odd static patterns that are common 
# ... A.A., AAS,e ends-with "/", etc etc
# "Applied Certificate in..." <--- thing is, this could really be a program
# the_df.STANDARDIZEDNAME_1 =\
#     the_df.STANDARDIZEDNAME_1.str.replace("A.A.","", case=False)



,NAME_1,STANDARDIZEDNAME_1
3494,Specialized Clinical Technician,Specialized Clinical Technician
14560,CXA-300-1I Advanced Administration for Citrix ...,CXA-300-1I Advanced Administration for Citrix ...
5535,Accounting Applications I/,Accounting Applications I/
20964,Project Management,Project Management
17769,LEED AP Exam,LEED AP Exam
22724,Medical Allied Health Office Assistant,Medical Allied Health Office Assistant
4720,Word Processing Program,Word Processing Program
12570,Certified Home Health Aide,Certified Home Health Aide
4186,Personal Trainer National Certification,Personal Trainer National Certification
2270,English as a Second Language,English as a Second Language


In [58]:
# Read in the acorynym list
# credential_lookup 
#path = rootpath + externalpath + CAREERONESTOP_CERTIFICATIONS_URL_NAME[1][:-4]
credential_sql = '2-CERTIFICATIONS.sql'

# note the faster alternative is to install a databse engine (seems to heavy handed)
# and read in form there.

# test string
my_string =\
    """
Insert into CERTIFICATIONS
   (CERT_ID, CERT_NAME, ORG_ID, TRAINING, EXPERIENCE, 
    EITHER, EXAM, RENEWAL, CEU, REEXAM, 
    CPD, CERT_ANY, URL, ACRONYM, NSSB_URL, 
    CERT_URL, CERT_LAST_UPDATE, KEYWORD1, KEYWORD2, KEYWORD3, 
    SUPPRESS, DATEADDED, COMMENTS, VERIFIED, UPDATEDBY, 
    CERT_DESCRIPTION, DELETED, EXAM_DETAILS)
 Values
   ('10249', 'SAP Certified Technology Associate - System Administration (Max DB) with SAP NetWeaver 7.4', '0106', NULL, NULL, 
    NULL, 1, '  ', NULL, NULL, 
    NULL, NULL, 'https://training.sap.com/certification/c_tadm50_74-sap-certified-technology-associate---system-administration-max-db-with-sap-netweaver-74-g/', NULL, NULL, 
    NULL, TO_DATE('01/28/2019 00:00:00', 'MM/DD/YYYY HH24:MI:SS'), NULL, NULL, NULL, 
    '1', TO_DATE('01/21/2015 16:31:10', 'MM/DD/YYYY HH24:MI:SS'), NULL, '0', '28', 
    'This certification path will validate your capability as a well-trained technologist prepared to help your client or partner manage SAP systems based on IBM DB2 database. Armed with an understanding of SAP system administration, you can implement this knowledge for your projects as a technology consultant', 1, NULL);
Insert into CERTIFICATIONS
   (CERT_ID, CERT_NAME, ORG_ID, TRAINING, EXPERIENCE, 
    EITHER, EXAM, RENEWAL, CEU, REEXAM, 
    CPD, CERT_ANY, URL, ACRONYM, NSSB_URL, 
    CERT_URL, CERT_LAST_UPDATE, KEYWORD1, KEYWORD2, KEYWORD3, 
    SUPPRESS, DATEADDED, COMMENTS, VERIFIED, UPDATEDBY, 
    CERT_DESCRIPTION, DELETED, EXAM_DETAILS)
 Values
   ('10052', 'IBM Certified BPM Developer - WebSphere Lombardi Edition V7.2', '0814', 1, 1, 
    NULL, 1, '  ', NULL, NULL, 
    NULL, NULL, 'http://www-03.ibm.com/certify/certs/15010502.shtml', NULL, NULL, 
    NULL, TO_DATE('07/06/2017 00:00:00', 'MM/DD/YYYY HH24:MI:SS'), 'Business process', NULL, NULL, 
    '1', TO_DATE('10/21/2014 10:43:08', 'MM/DD/YYYY HH24:MI:SS'), NULL, '0', '28', 
    'This intermediate level certification is intended for BPM application developers who contribute to the delivery of complex level process applications, and provide supervision and guidance to entry level developers.', 1, NULL);
    """

test_field =\
    """
'10052', 'IBM Certified BPM Developer - WebSphere Lombardi Edition V7.2', '0814', 1, 1, 
    NULL, 1, '  ', NULL, NULL, 
    NULL, NULL, 'http://www-03.ibm.com/certify/certs/15010502.shtml', NULL, NULL, 
    NULL, TO_DATE('07/06/2017 00:00:00', 'MM/DD/YYYY HH24:MI:SS'), 'Business process', NULL, NULL, 
    '1', TO_DATE('10/21/2014 10:43:08', 'MM/DD/YYYY HH24:MI:SS'), NULL, '0', '28', 
    'This intermediate level certification is intended for BPM application developers who contribute to the delivery of complex level process applications, and provide supervision and guidance to entry level developers.', 1, NULL    
    """

header =\
    ('CERT_ID', 'CERT_NAME', 'ORG_ID', 'TRAINING', 'EXPERIENCE', 
    'EITHER', 'EXAM', 'RENEWAL', 'CEU', 'REEXAM', 
    'CPD', 'CERT_ANY', 'URL', 'ACRONYM', 'NSSB_URL', 
    'CERT_URL', 'CERT_LAST_UPDATE', 'KEYWORD1', 'KEYWORD2', 'KEYWORD3', 
    'SUPPRESS', 'DATEADDED', 'COMMENTS', 'VERIFIED', 'UPDATEDBY', 
    'CERT_DESCRIPTION', 'DELETED', 'EXAM_DETAILS')

the_fields_regex =\
    """
    (?P<values>Values\n\s+\()  # Start with the word Value <newline> (
        (?P<fields>.*?)        #    Grab all the field content
    (?P<end>\);)               # ... which stops at the terminating paren, ;
    """

flags = re.MULTILINE | re.DOTALL | re.VERBOSE
the_fields = re.compile(the_fields_regex, flags=flags)

a_field_regex =\
    """
 '(?P<quoted_string>.*?)'       # get a quoted string or
|(?P<date_time>TO_DATE\(.*?\))  # get the TO_DATE, parse out actual date later or
|(?P<num>\d),                   # get numeric or
|(?P<null>NULL)                 # get NULL
    """

a_field = re.compile(a_field_regex, flags=flags)

groups = {1: 'date_time', 2: 'quoted_string', 3: 'num', 4:'null'}

for match in the_fields.finditer(my_string):
    for index, field in enumerate(a_field.finditer( match.group('fields') )):
        grp = None
        for grp, value in field.groupdict().items():
            if value:
                if grp == 'date_time':
                    #  There is a difference between https://regex101.com/r/yphUXY/1/
                    # and what I see Python do here; if I don't capture the entire thing
                    # it gets re-raised as another potential match, even if I use ?:, etc.
                    value = value[9:28]
                grp = grp
                break

        print(index, grp, header[index], value)

0 quoted_string CERT_ID 10249
1 quoted_string CERT_NAME SAP Certified Technology Associate - System Administration (Max DB) with SAP NetWeaver 7.4
2 quoted_string ORG_ID 0106
3 null TRAINING NULL
4 null EXPERIENCE NULL
5 null EITHER NULL
6 num EXAM 1
7 quoted_string RENEWAL   
8 null CEU NULL
9 null REEXAM NULL
10 null CPD NULL
11 null CERT_ANY NULL
12 quoted_string URL https://training.sap.com/certification/c_tadm50_74-sap-certified-technology-associate---system-administration-max-db-with-sap-netweaver-74-g/
13 null ACRONYM NULL
14 null NSSB_URL NULL
15 null CERT_URL NULL
16 date_time CERT_LAST_UPDATE 01/28/2019 00:00:00
17 null KEYWORD1 NULL
18 null KEYWORD2 NULL
19 null KEYWORD3 NULL
20 quoted_string SUPPRESS 1
21 date_time DATEADDED 01/21/2015 16:31:10
22 null COMMENTS NULL
23 quoted_string VERIFIED 0
24 quoted_string UPDATEDBY 28
25 quoted_string CERT_DESCRIPTION This certification path will validate your capability as a well-trained technologist prepared to help your client or pa

In [42]:
another_string =\
"""
    NULL, TO_DATE('01/28/2019 00:00:00', 'MM/DD/YYYY HH24:MI:SS'), NULL, NULL, NULL, 
"""
# (?P<date_time>TO_DATE\(.*?'MM/DD/YYYY HH24:MI:SS\'\))
regex = """
 (?P<date_time>TO_DATE\(.*\)),
|'(?P<quoted_string>.*?)'
|(?P<num>\d),
|(?P<null>NULL)
"""

for index, field in enumerate(re.finditer(regex, another_string, flags=flags )):
    for grp, value in field.groupdict().items():
        if value:
            print(grp)

    print(index, header[index], field)

null
0 CERT_ID <re.Match object; span=(5, 9), match='NULL'>
date_time
1 CERT_NAME <re.Match object; span=(11, 67), match="TO_DATE('01/28/2019 00:00:00', 'MM/DD/YYYY HH24:M>
null
2 ORG_ID <re.Match object; span=(68, 72), match='NULL'>
null
3 TRAINING <re.Match object; span=(74, 78), match='NULL'>
null
4 EXPERIENCE <re.Match object; span=(80, 84), match='NULL'>


In [ ]:
#  now with the transformed data we 
# a) launch an active label checker, that looks for program/course names that are standardized or not standardized
# b) periodically run the cell below that calcualtes the bionomial proportion of standardized to unstandardized; ideally we want 95% - 100%

In [6]:
# insert active labeler here
# this is crude but first we set up the pipeline, following
# https://superintendent.readthedocs.io/en/latest/examples/preprocessing-data.html
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import ComplementNB  # corrects for class imbalance, SGD is pretty good too
from sklearn.pipeline import Pipeline

from superintendent import ClassLabeller
from IPython.display import display, Markdown

pipeline = Pipeline([
    ('vect', CountVectorizer(analyzer='char', ngram_range=(1,2))),
    ('tfidf', TfidfTransformer()),
    ('clf', ComplementNB()),
])

def display_func(row):
    """
    The display function gets passed your data - in the
    case of a dataframe, it gets passed a row - and then
    has to "display" your data in whatever way you want.

    It doesn't need to return anything
    """
    display(Markdown(row["STANDARDIZEDNAME_1"]))
    #display(Markdown("**At:** " + row["timestamp"]))

def preprocessor(x, y):
    # only take standardized column, leave everything else
    return x["STANDARDIZEDNAME_1"], y

labelling_widget = ClassLabeller(
    features=the_df,
    model=pipeline,
    model_preprocess=preprocessor,
    display_func=display_func,
    options=['standardized', 'not standardized'],
    acquisition_function='margin'
)

labelling_widget

ClassLabeller(children=(HBox(children=(HBox(children=(FloatProgress(value=0.0, description='Progress:', max=1.…

In [26]:
# insert bionomial proprtion esimator here

def print_CI(labels, response_is_standardized = "standardized", method = "jeffreys"):
    successful_count = sum(
        response_is_standardized == label for label in labels
    )
    not_examined_count = sum(
        None == label for label in labels
    )

    CI = proportion_confint(
            count= successful_count,
            nobs= len(labels) - not_examined_count,
            alpha = 0.95,
            method=method
        )
    print(f"{method} bionomial proportion is: [{CI[0]:.2f}, {CI[1]:.2f}]",
)
print_CI(labels=labelling_widget.new_labels)

jeffreys bionomial proportion is: [0.88, 0.88]


In [142]:
# ... finally we can write this out as our first complete lookup table
# for the NAME field
the_df.to_csv(rootpath + processedpath + "{}".format(content_is),
              index = False,
              chunksize = 10000,
              columns=columns_to_save)